In [18]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [19]:
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

#### 1 - Loading the documents

In [11]:
from langchain_community.document_loaders import DirectoryLoader

directory = "../data/raw"


def load_docs(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents


documents = load_docs(directory)
len(documents)

2

#### 2 - Splitting the documents

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


def split_docs(documents, chunk_size=500, chunk_overlap=20):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    chunks = splitter.split_documents(documents)
    return chunks


docs = split_docs(documents)
len(docs)

8

#### 3 - Embedding the documents

In [28]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

d:\Softwares\anaconda\envs\llm-data-analyst\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\win 11\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [25]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
existing_indexes

['ai-assistant', 'langchain-demo']

In [26]:
index_name = "ai-assistant"
index = pc.Index(index_name)

In [29]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

In [32]:
def get_similar_docs(query, k=2, score=False):
    if score:
        similar_docs = docsearch.similarity_search_with_score(query, k=k)
    else:
        similar_docs = docsearch.similarity_search(query, k=k)
    return similar_docs

In [33]:
query = "Who took over twitter?"
similar_docs = get_similar_docs(query)
print(similar_docs)

[Document(metadata={'source': '..\\data\\raw\\twitter_data.pdf'}, page_content='Twitter was founded in 2006 and was listed on the stock exchange in 2013. Since the founding of Twitter, 2022 has been an event to remember Twitter. As Elon Musk took over Twitter, it will be delisted from the New York Exchange. As 2022 was so eventful for Twitter, analyze the complete timeline of Twitter in the Stock Market from 2013 to 2022. Twitter is one of the popular social media applications where people share what they feel in a limited number of words. Twitter is popular but not in the'), Document(metadata={'source': '..\\data\\raw\\elon.txt'}, page_content='Elon Musk is known for his ambitious goals, innovative thinking, and willingness to take on complex challenges. His work has had a profound impact on the automotive, aerospace, and renewable energy industries. Musk is also known for his outspoken presence on social media, where he shares updates on his companies, thoughts on technology and the 